Copyright © 2020 IUBH Internationale Hochschule

#### Bayessche Statistik Interaktiv

Der Satz von Bayes wird hier anhand eines Besipiels gezeigt. Einzelne Wahrscheinlichkeiten können interaktiv geändert werden wodurch die Änderungen am Endergebnnis sofort sichtbar sind.

Erläuterungen zum Beispiel:

- Als Beispiel soll ein medizinischer Test gewahlt werden. Herauszufinden gilt (a posteriori Wahrscheinlichkeit):
 * Ist ein Patient erkrankt, wenn der Test positiv anschlägt?

Interaktive Eingaben:
- Wenn der Patient erkrankt ist, schlaegt der Test mit einer Wahrscheinlichkeit von 0.xx an. (voreingestellt: 0.99)
- in x-Fällen ist der Test positiv, auch wenn der Patient nicht erkrankt ist (voreingestellt: 0.005)
- Prior Information, d.h. wie weit ist die Krankheit verbreitet (voreingestellt: 0.0001)

Daraus werden zunaechst die bedingten Wahrscheinlichkeiten berechnet:
- P(krank)
- P(nicht krank)
- P(test positiv|erkrankt)
- P(test positiv | nicht erkrankt)

Und dann mittels des Bayesschen Theorems P(krank|test positiv) - was wir wissen wollen.


Theorem von Bayes:
Für zwei Ereignisse A und B, für B ≠ 0, lautet das Satz von Bayes:

$$ {P(A|B)} = \frac{P(B|A) \bullet P(A)}{P(B)}$$

P(A | B) ist die (bedingte) Wahrscheinlichkeit des Ereignisses A unter der Bedingung, dass B eingetreten ist. P(B | A) ist die (bedingte) Wahrscheinlichkeit des Ereignisses B unter der Bedingung, dass A eingetreten ist
P(A) ist die Wahrscheinlichkeit (Anfangswahrscheinlichkeit) für das Eintreten des Ereignisses A
P(B) ist die Wahrscheinlichkeit (Anfangswahrscheinlichkeit) für das Eintreten des Ereignisses B
Anfangswahrscheinlichkeit meint, dass ein Ereignis unabhängig von einem anderen betrachtet wird.

#### Bayesian Statistics Interactive

Bayes' theorem is shown here using an example. Individual probabilities can be changed interactively so that the changes to the end result are immediately visible.

Explanations for example:

- As an example, a medical test should be chosen. Find out (a posteriori probability):
 * Is a patient ill when the test is positive?

Interactive inputs:
- If the patient is diseased, the test has a probability of 0.xx. (default: 0.99)
- in x cases the test is positive, even if the patient is not diseased (default: 0.005)
- Prior information, i.e. how widespread the disease is (default: 0.0001)

The conditional probabilities are first calculated from this:
- P(diseased)
- P(not diseased)
- P(test positive | diseased)
- P(test positive | not diseased)

And then by means of the Bayesian theorem P(diseased | test positive) - what we want to know.


Bayes' theorem:
For two events A and B, for B ≠ 0, Bayes' theorem is:

$$ {P(A|B)} = \frac{P(B|A) \bullet P(A)}{P(B)}$$

P(A | B) is the (conditional) probability of event A under the condition that B has occurred. P(B | A) is the (conditional) probability of event B under the condition that A has occurred
P(A) is the probability (initial probability) that event A will occur
P(B) is the probability (initial probability) that event B will occur
Initial probability means that one event is viewed independently of another.

#### Bayesian Statistics Interactive

Bayes' theorem is shown here using an example. Individual probabilities can be changed interactively so that the changes to the end result are immediately visible.

Explanations for example:

- As an example, a medical test should be chosen. Find out (a posteriori probability):
 * Is a patient ill when the test is positive?

Interactive inputs:
- If the patient is ill, there test has a probability of 0.xx. (default: 0.99)
- in x cases the test is positive, even if the patient is not ill (default: 0.005)
- Prior information, i.e. how widespread the disease is (default: 0.0001)

The conditional probabilities are first calculated from this:
- P(diseased)
- P(not diseased)
- P(test positive | diseased)
- P(test positive | not diseased)

And then by means of the Bayesian theorem P (diseased | test positive) - what we want to know.


Bayes' theorem:
For two events A and B, for B ≠ 0, Bayes' theorem is:

$$ {P(A|B)} = \frac{P(B|A) \bullet P(A)}{P(B)}$$

P (A undefined B) is the (conditional) probability of event A under the condition that B has occurred. P (B | A) is the (conditional) probability of event B under the condition that A has occurred
P (A) is the probability (initial probability) that event A will occur
P (B) is the probability (initial probability) that event B will occur
Initial probability means that one event is viewed independently of another.

#### Bayessche Statistik Interaktiv

Der Satz von Bayes wird hier anhand eines Besipiels gezeigt. Einzelne Wahrscheinlichkeiten können interaktiv geändert werden wodurch die Änderungen am Endergebnnis sofort sichtbar sind.

Erläuterungen zum Beispiel:

- Als Beispiel soll ein medizinischer Test gewahlt werden. Herauszufinden doré (a posteriori Wahrscheinlichkeit):
 * Ist ein Patient erkrankt, wenn der Test positiv anschlägt?

Interaktive Eingaben:
- Wenn der Patient erkrankt ist, schlaegt der Test mit einer Wahrscheinlichkeit von 0.xx an. (voreingestellt: 0.99)
- dans x-Fällen ist der Test positiv, auch wenn der Patient nicht erkrankt ist (voreingestellt: 0,005)
- Informations préalables, d.h. wie weit ist die Krankheit verbreitet (voreingestellt: 0.0001)

Daraus werden zunaechst die bedingten Wahrscheinlichkeiten berechnet:
- P (krank)
- P (nicht krank)
- P (test positivundefinederkrankt)
- P (test positif non défini nicht erkrankt)

Und dann mittels des Bayesschen Theorems P (krankundefinedtest positiv) - était wir wissen wollen.


Théorème de Bayes:
Für zwei Ereignisse A und B, für B ≠ 0, lautet das Satz von Bayes:

$$ {P(A|B)} = \frac{P(B|A) \bullet P(A)}{P(B)}$$

P (A undefined B) ist die (bedingte) Wahrscheinlichkeit des Ereignisses A unter der Bedingung, dass B eingetreten ist. P (B | A) ist die (bedingte) Wahrscheinlichkeit des Ereignisses B unter der Bedingung, dass A eingetreten ist
P (A) ist die Wahrscheinlichkeit (Anfangswahrscheinlichkeit) für das Eintreten des Ereignisses A
P (B) ist die Wahrscheinlichkeit (Anfangswahrscheinlichkeit) für das Eintreten des Ereignisses B
Anfangswahrscheinlichkeit meint, dass ein Ereignis unabhängig von einem anderen betrachtet wird.

In [ ]:
#resourcen

import ipywidgets as widgets
import numpy as np


In [ ]:
# interactive switches

# Reliability of detection if patient falls ill
selectionValues = np.arange(0.0001, 1, 0.0001)
#
pB_A = widgets.SelectionSlider( options=selectionValues, value=0.99, orientation='horizontal',\
                               readout=True, layout={'width': 'max-content'})

# Proportion of positive tests in healthy patients
pB_Anot = widgets.SelectionSlider( options=selectionValues, value=0.005, orientation='horizontal',\
                               readout=True, layout={'width': 'max-content'}, style = {'description_width': 'initial'})
#
pA = widgets.SelectionSlider( options=selectionValues, value=selectionValues[0], orientation='horizontal',\
                               readout=True, layout={'width': '20%'})
#
pVerbreitung = widgets.HBox(
    [
        widgets.Label(value="Spread of the disease:"),
        pA
    ]
)
#
pNachweis = widgets.VBox(
    [
        widgets.Label(
            value='Verifiability of the test when the patient is ill:'),
        pB_A        
    ]
)
#
pFehler = widgets.VBox(
    [
        widgets.Label(value='Error rate of the test if the patient is not ill:'),
        pB_Anot
    ]
)
#
style = {'description_width': 'initial'}
outpriori = widgets.VBox(
[
    # P(ill) = pA
    widgets.Box([widgets.Label(value="P(ill)", layout={'width': '30%'}),
                widgets.FloatText(description = "", value = pA.value ,\
                                  layout={'width': '20%'}, disabled = True,style=style)]
                ),
    # P(not ill) = 1-pA
    widgets.Box([widgets.Label(value="P(not ill)", layout={'width': '30%'}),
                 widgets.FloatText(value = ((1 - pA.value/100)),\
                                   layout ={'width': '20%'}, disabled = True,style=style)]
               ),
    # P (test _positive | ill) = pB_ A (p detection reliability)
    widgets.Box([widgets.Label(value= "P(test _positive | ill)", layout={'width': '30%'}),
                 widgets.FloatText(value = '%1.6f' % (pB_A.value),\
                                   layout={'width': '20%'}, disabled = True,style=style)]
               ),
    # P (test _positive | not ill) = pB_ anot (positive test in healthy patient)
    widgets.Box([widgets.Label(value="P(test _positive | not ill)", layout={'width': '30%'}),
                 widgets.FloatText(value = '%1.6f' %(pB_Anot.value),\
                                   layout={'width': '20%'}, disabled = True, style=style)]
               )
])
# P (ill | test _positive) = [pA * p detection security] / [pA * p detection security + pA_ not *p error rate] {= pB over total probability}
outposteriori = widgets.VBox(
[
    widgets.Label(value="Probability that a patient will fall ill if the test is positive:"),
    widgets.FloatText(value = '%1.6f' % (pB_A.value * pA.value / \
                                         (pB_A.value * pA.value + pB_Anot.value * (1 - pA.value))),\
                      description="", disabled = True, layout={'width': '30%'})
])
#
BtnReset = widgets.Button(description="reset", button_style='warning')
#

In [ ]:
# update function that is executed when a switch is changed
#
def update_view(*args):
    """                                      _
             p(B|A) = 0.99      p (B|A) = 0.05
            /                 _/
        p(A)                p(A)
            \   _              \  _ _
             p (B|A) = 0.01     p(B|A) = 0.95
             
        P (diseased) = pA
        P (not sick) = 1-pA
        P (test _positive | ill) = pB_ A (default: 0.99)
            {P (test _negative | diseased) = 1- pB_ A (default: 1 - 0.99)} not shown
        P (test _positive | not ill) = pB_ Anot (default: 0.05)
            {P (test _negative | not sick) = 1 - pB_ Anot (default: 1- 0.05)} not shown
        P (ill | test _positive) = [pB_ A] *pA / [pB_A* pA + pB _Anot * 1-pA] {=> P (B) calculated over total probability }
        
        """
    outpriori.children[0].children[1].value = pA.value
    outpriori.children[1].children[1].value = (1 - pA.value)
    outpriori.children[2].children[1].value = '%1.6f' % (pB_A.value)
    outpriori.children[3].children[1].value = '%1.6f' %(pB_Anot.value)
    #
    # Intercept extreme values
    if (pB_Anot.value == 0):
        outposteriori.children[1].value = 0.0
        return
    if (pA.value == 100):
        outposteriori.children[1].value = 100
        return
    outposteriori.children[1].value = '%1.6f' % ((pB_A.value * pA.value) / \
                                                 (pB_A.value * pA.value + pB_Anot.value * (1-pA.value)))
    #

In [ ]:
# Reset settings to default values
#
def reset (* btn):
    pA.value = 0.0001
    pB_A.value = 0.99
    pB_Anot.value= 0.005
    update_view()

In [ ]:
# Add update function
#
pA.observe(update_view, 'value')
pB_A.observe(update_view, 'value')
pB_Anot.observe(update_view, 'value')
BtnReset.on_click(reset)
#
# Arrange switch
#
widgets.VBox([widgets.VBox([pVerbreitung, pNachweis, pFehler]),
             outpriori,
             outposteriori,
             BtnReset])

Copyright © 2020 IUBH Internationale Hochschule